# STATISTICS Applied to data science

## Exercises PART 4: Model Evaluation with statistical tests

Model Evaluation is a crucial part of the **Proof-of-concept** stage. It also usually takes quite a long time because it's mainly a trial-and-error cycle. You start with the simplest model and iterate through many kinds of analysis until you can say you found the best model possible.

**Simple question during model evaluation:** 
I'm training a model to predict the daily sales of PARFUM and MAKE_UP.  
The overall error (I chose the metric MAE) of the model is 305 orders per day. But when analized the MAE by clusters, I got 270 (s.d.=23) for PARFUM and 338 (s.d.=80) for MAKE-UP. Is this model statistically performing worse for MAKE_UP? How do you answer this question? 

![Image](../images/data_1.jpg)

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from statsmodels.stats import power
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (12, 8)
%matplotlib inline
# jupyter lab configs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.float_format', lambda x: '%.2f' % x)

## Get familiar with Statsmodels

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

Get a list of available built-in datasets:

In [ ]:
dir(sm.datasets)

# Example of model evaluation 

### We'll use the Rossman dataset

In [ ]:
sales = pd.read_csv('../../timeseries predictive modelling/datasets/rossman_train.csv')
stores = pd.read_csv('../../timeseries predictive modelling/datasets/rossman_store.csv')

# join store features into the sales df
sales = pd.merge(sales, stores, on='Store', how='left')

# a bit of cleaning
sales = sales[(sales.Open==1)&(sales.Sales>0)]

In [ ]:
sales.head(4)

In [ ]:
len(sales)

In [ ]:
sales.groupby(['Store']).size()

In [ ]:
test = pd.read_csv('datasets/rossman_test.csv')
test['Date'].min(), test['Date'].max()

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics

sales.dropna(inplace=True)
X = sales[['DayOfWeek', 'StoreType', 'Assortment', 'CompetitionDistance', 'Promo2', 'Promo']].copy()
X = pd.concat([X, pd.get_dummies(X[['StoreType', 'Assortment']], prefix=['type', 'assortment'])], axis=1)
X.drop(columns=['StoreType', 'Assortment'], inplace=True)
X.head()

y = sales.Sales.copy()

In [ ]:
y.describe()

In [ ]:
X.columns

In [ ]:
#X = X[['CompetitionDistance','Promo', 'Promo2', 'type_a', 'type_c', 'type_d', 'assortment_a', 'assortment_c']]
X = X[[ 'Promo', 'type_a', 'type_c', 'type_d', 'assortment_a', 'assortment_c']]
y = sales.Sales

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

reg = LinearRegression().fit(X_train, y_train)
y_pred = reg.predict(X_test)


# Regression metrics
metrics.mean_absolute_error(y_test, y_pred) 
#metrics.mean_squared_error(y_test, y_pred) 
#metrics.r2_score(y_test, y_pred)

Let's look in more detail 

In [ ]:
errors_df = pd.concat([X_test, y_test, pd.Series(y_pred, index=y_test.index, name='y_pred')], axis=1)
errors_df['Abs_error'] = np.absolute(errors_df.Sales - errors_df.y_pred)

agg_dict = {'Abs_error' : ['mean', 'std']}

In [ ]:
errors_df

In [ ]:
errors_df.groupby([ 'Promo',  'type_a', 'type_c', 'type_d', 'assortment_a', 'assortment_c']).agg(agg_dict).reset_index()

### We can use a T-test to compare sales of stores with and without the `promo` flag:

In [ ]:
groupa = errors_df[(errors_df.assortment_a==1)&(errors_df.type_d==1)&(errors_df.Promo==0)].Abs_error
groupb = errors_df[(errors_df.assortment_a==1)&(errors_df.type_a==1)&(errors_df.Promo==1)].Abs_error

In [ ]:
groupa.mean(), groupb.mean()

In [ ]:
stats.ttest_ind(groupa, groupb, equal_var=False)

---

## Use a toy dataset with multiple features to fit a linear model and evaluate it

### Let's use the Boston dataset from sklearn and try to predict house prices depending on features of the city

In [ ]:
from sklearn.datasets import load_boston

# Load Boston house prices data
dt = load_boston(return_X_y=False)
df = pd.DataFrame(data = np.c_[dt['data'],dt['target']])
df.columns = np.append(dt['feature_names'], 'MED_VALUE')
df.drop(['B', 'LSTAT'], inplace=True, axis=1)
df.head()

## Description of this dataset

**Features** 

CRIM = per capita crime rate by town  
ZN = proportion of residential land zoned for lots over 25,000 sq.ft.  
INDUS = proportion of non-retail business acres per town  
CHAS = Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)  
NOX = nitric oxides concentration (parts per 10 million)  
RM = average number of rooms per dwelling  
AGE = proportion of owner-occupied units built prior to 1940  
DIS = weighted distances to five Boston employment centres  
RAD = index of accessibility to radial highways  
TAX = full-value property-tax rate per $10,000  
PTRATIO = pupil-teacher ratio by town   

**Target**

MEDV Median value of owner-occupied homes in $1000’s

In [ ]:
df.head(4)

In [ ]:
len(df)

### Example of model fitting with `Stats_models`

In [ ]:
# use formula-like strings to specify your model
mod = smf.ols(formula='MED_VALUE ~ CRIM+ZN+INDUS+RM+DIS+PTRATIO', data=df) 
# fit
res = mod.fit()            
# summarize
print(res.summary())                        

### Aren't we forgetting an important step? 
How about some scaling?

## 2. Pre-processing 

Check if you have missing data, outliers, typos

## 3. Visualizations  

The only way to understand the data. I recomend using **Plotly for Python**.  

You can see some other nice plots of this dataset here: http://www.neural.cz/dataset-exploration-boston-house-pricing.html

## 4. Is the data ok? Then split into train and test datasets

You can use `train_test_split()` from **SkLearn**

In [ ]:
from sklearn.model_selection import train_test_split

## 5. Fit the model to the training data

## 6. Predict using the test data

## 7. Calculate error metrics
You could use MAE and RMSE, for example. Calculate the errors of your training and test data. Is your model overfitting or underfitting?  

The question you should ask now is, *is my model performing the same across all segments of my data?*   
To answer that, separate your data into clusters (for example `CHAS`, or number of rooms `RM`) and calculate the average error and standar deviation of the error per cluster. Are they the same? Do they differ? If they differ, is this difference significant?

#### What if most of the features are quantitative?  

In this case you can always analyze the error per feature, with correlations, for example. Is the error maybe higher in the lower or higher ends of a given feature?  
Another solution is to simplify and cluster the data. One solution is to use `cut()` from Pandas

Let's use the feature `DIS`, which express distance to employement centers. Can we create maybe three bins, `SHORT`, `MEDIUM`, `LONG`?

In [ ]:
df.DIS.describe()

Ok, the distance ranges from 1.13 to 12.13. We can create 3 bins in the following way: 

In [ ]:
bins = [0, 4, 8, 13]
labels = ['SHORT', 'MEDIUM', 'LONG']
df['DIS_CAT'] = pd.cut(df['DIS'], bins=bins, labels=labels)
print(df)